In [ ]:
import pickle
import numpy as np
from scipy.signal import savgol_filter, medfilt
from pywt import wavedec, waverec, threshold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Global dictionary to store feature names and values
feature_dict = {}

def add_feature(name, func, *args):
    """Try to add a feature by checking the shape and ensuring it’s a scalar."""
    try:
        value = func(*args)
        if np.isscalar(value):
            feature_dict[name] = value
        elif isinstance(value, (list, tuple, np.ndarray)) and value.size == 1:
            feature_dict[name] = value.item()
        else:
            print(f"Warning: Feature '{name}' has incorrect shape and was not added.")
    except Exception as e:
        print(f"Error computing feature '{name}': {e}")

# Denoising algorithms
def wavelet_denoise(signal, wavelet='db1', level=1, thresholding='soft'):
    coeffs = wavedec(signal, wavelet, level=level)
    coeffs[1:] = [threshold(c, np.median(np.abs(c)) / 0.6745, mode=thresholding) for c in coeffs[1:]]
    return waverec(coeffs, wavelet)

def fourier_denoise(signal, cutoff_freq):
    fft_signal = np.fft.fft(signal)
    freqs = np.fft.fftfreq(len(signal))
    fft_signal[np.abs(freqs) > cutoff_freq] = 0
    return np.fft.ifft(fft_signal).real

def savgol_denoise(signal, window_length=5, polyorder=2):
    return savgol_filter(signal, window_length, polyorder)

def median_denoise(signal, kernel_size=3):
    return medfilt(signal, kernel_size)

def moving_average_denoise(signal, window_size=5):
    return np.convolve(signal, np.ones(window_size) / window_size, mode='same')

# Apply denoising and extract features
def apply_denoising_and_extract_features(data, denoise_func, *denoise_args):
    features = []
    labels = []
    snrs = []
    
    for key, signals in data.items():
        mod_type, snr = key
        for signal in signals:
            real_part, imag_part = signal[0], signal[1]
            
            # Apply denoising to real and imaginary parts separately
            real_denoised = denoise_func(real_part, *denoise_args)
            imag_denoised = denoise_func(imag_part, *denoise_args)
            
            # Combine real and imaginary into magnitude and phase
            magnitude = np.sqrt(real_denoised**2 + imag_denoised**2)
            phase = np.arctan2(imag_denoised, real_denoised)
            
            # Reset feature dictionary for each signal
            global feature_dict
            feature_dict = {}

            # Add features using magnitude and phase
            add_feature("Magnitude Mean", np.mean, magnitude)
            add_feature("Magnitude Variance", np.var, magnitude)
            add_feature("Phase Mean", np.mean, phase)
            add_feature("Phase Variance", np.var, phase)
            add_feature("Real Part Mean", np.mean, real_denoised)
            add_feature("Imag Part Mean", np.mean, imag_denoised)
            add_feature("Real Part Variance", np.var, real_denoised)
            add_feature("Imag Part Variance", np.var, imag_denoised)

            # Add SNR as a feature
            feature_dict["SNR"] = snr  # Include SNR as part of the features

            # Append the feature values and label
            features.append(list(feature_dict.values()))
            labels.append(mod_type)

    return np.array(features), labels

# Load the RML2016.10a_dict.pkl file with explicit encoding
with open("../RML2016.10a_dict.pkl", "rb") as f:
    data = pickle.load(f, encoding="latin1")

# Denoising methods to test
denoise_methods = {
    "Wavelet Denoising": (wavelet_denoise, ('db1', 1, 'soft')),
    "Fourier Denoising": (fourier_denoise, (0.1,)),
    "Savitzky-Golay Filter": (savgol_denoise, (5, 2)),
    "Median Filter": (median_denoise, (3,)),
    "Moving Average Filter": (moving_average_denoise, (5,))
}

# Train and evaluate classifier for each denoising method
for method_name, (denoise_func, denoise_args) in denoise_methods.items():
    print(f"\nEvaluating {method_name}")
    
    # Extract features using denoising
    features, labels = apply_denoising_and_extract_features(data, denoise_func, *denoise_args)
    
    # Encode labels for classification
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    
    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.3, random_state=42)
    
    # Train classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    
    # Evaluate accuracy
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy with {method_name}: {accuracy * 100:.2f}%")

    # Plot confusion matrix for each method
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
                xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(f"Confusion Matrix for {method_name}")
    plt.show()


In [ ]:
import pickle
import numpy as np
from scipy.signal import savgol_filter, medfilt, wiener
from scipy.ndimage import gaussian_filter1d
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Global dictionary to store feature names and values
feature_dict = {}

def add_feature(name, func, *args):
    """Try to add a feature by checking the shape and ensuring it’s a scalar."""
    try:
        value = func(*args)
        if np.isscalar(value):
            feature_dict[name] = value
        elif isinstance(value, (list, tuple, np.ndarray)) and value.size == 1:
            feature_dict[name] = value.item()
        else:
            print(f"Warning: Feature '{name}' has incorrect shape and was not added.")
    except Exception as e:
        print(f"Error computing feature '{name}': {e}")

# Denoising algorithms
def ema_denoise(signal, alpha):
    ema_signal = np.zeros_like(signal)
    ema_signal[0] = signal[0]
    for i in range(1, len(signal)):
        ema_signal[i] = alpha * signal[i] + (1 - alpha) * ema_signal[i - 1]
    return ema_signal

def gaussian_denoise(signal, sigma):
    return gaussian_filter1d(signal, sigma)

def wiener_denoise(signal, mysize=5):
    return wiener(signal, mysize)

# Apply denoising and extract features
def apply_denoising_and_extract_features(data, denoise_func, *denoise_args):
    features = []
    labels = []
    snrs = []
    
    for key, signals in data.items():
        mod_type, snr = key
        for signal in signals:
            real_part, imag_part = signal[0], signal[1]
            
            # Apply denoising to real and imaginary parts separately
            real_denoised = denoise_func(real_part, *denoise_args)
            imag_denoised = denoise_func(imag_part, *denoise_args)
            
            # Combine real and imaginary into magnitude and phase
            magnitude = np.sqrt(real_denoised**2 + imag_denoised**2)
            phase = np.arctan2(imag_denoised, real_denoised)
            
            # Reset feature dictionary for each signal
            global feature_dict
            feature_dict = {}

            # Add features using magnitude and phase
            add_feature("Magnitude Mean", np.mean, magnitude)
            add_feature("Magnitude Variance", np.var, magnitude)
            add_feature("Phase Mean", np.mean, phase)
            add_feature("Phase Variance", np.var, phase)
            add_feature("Real Part Mean", np.mean, real_denoised)
            add_feature("Imag Part Mean", np.mean, imag_denoised)
            add_feature("Real Part Variance", np.var, real_denoised)
            add_feature("Imag Part Variance", np.var, imag_denoised)

            # Add SNR as a feature
            feature_dict["SNR"] = snr  # Include SNR as part of the features

            # Append the feature values and label
            features.append(list(feature_dict.values()))
            labels.append(mod_type)

    return np.array(features), labels

# Load the RML2016.10a_dict.pkl file with explicit encoding
with open("../RML2016.10a_dict.pkl", "rb") as f:
    data = pickle.load(f, encoding="latin1")

# Denoising methods and parameter values to test
denoise_methods = {
    "EMA Alpha=0.1": (ema_denoise, (0.1,)),
    "EMA Alpha=0.3": (ema_denoise, (0.3,)),
    "EMA Alpha=0.5": (ema_denoise, (0.5,)),
    "Gaussian Sigma=1": (gaussian_denoise, (1,)),
    "Gaussian Sigma=2": (gaussian_denoise, (2,)),
    "Wiener Filter": (wiener_denoise, (5,))
}

# Train and evaluate classifier for each denoising method
for method_name, (denoise_func, denoise_args) in denoise_methods.items():
    print(f"\nEvaluating {method_name}")
    
    # Extract features using denoising
    features, labels = apply_denoising_and_extract_features(data, denoise_func, *denoise_args)
    
    # Encode labels for classification
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    
    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.3, random_state=42)
    
    # Train classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    
    # Evaluate accuracy
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy with {method_name}: {accuracy * 100:.2f}%")

    # Plot confusion matrix for each method
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
                xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(f"Confusion Matrix for {method_name}")
    plt.show()


In [ ]:

def moving_average_denoise(signal, window_size=10):
    return np.convolve(signal, np.ones(window_size) / window_size, mode='same')

# Denoising methods and parameter values to test
denoise_methods = {
    "Moving Average Filter": (moving_average_denoise, (10,))
}

# Train and evaluate classifier for each denoising method
for method_name, (denoise_func, denoise_args) in denoise_methods.items():
    print(f"\nEvaluating {method_name}")
    
    # Extract features using denoising
    features, labels = apply_denoising_and_extract_features(data, denoise_func, *denoise_args)
    
    # Encode labels for classification
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    
    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.3, random_state=42)
    
    # Train classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    
    # Evaluate accuracy
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy with {method_name}: {accuracy * 100:.2f}%")

    # Plot confusion matrix for each method
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
                xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(f"Confusion Matrix for {method_name}")
    plt.show()
